# INITIALISATION

In [84]:
countries = [
    "Brazil",
    "Spain",
    "France",
    "Argentina",
    "Uruguay",
    "Colombia",
    "United Kingdom",
    "Paraguay",
    "Germany",
    "Ecuador"
]

elo_ratings = [
    1994,
    2150,
    2031,
    2140,
    1922,
    1953,
    2012,
    1799,
    1988,
    1911,
]

def get_elo(name):
    return countriesratings[name]

countriesratings = {country: elo_ratings[countries.index(country)] for country in countries}
countries_ranked = sorted(countries, key=get_elo, reverse=True)

locations = [
    (-14.2350, -51.9253),  # Brazil
    (40.4637, -3.7492),    # Spain
    (46.6034, 1.8883),     # France
    (-38.4161, -63.6167),  # Argentina
    (-32.5228, -55.7659),  # Uruguay
    (4.5709, -74.2973),    # Colombia
    (55.3781, -3.4360),    # United Kingdom
    (-23.4420, -58.4438),  # Paraguay
    (51.1657, 10.4515),    # Germany
    (-1.8312, -78.1834),   # Ecuador
]

locationdict = {country: locations[countries.index(country)] for country in countries}

# DISTANCE CALCULATION

In [85]:
from math import radians, sin, cos, sqrt, atan2

# DISTANCE CALCULATIONS
def haversine(pos1, pos2):
    lat1, lon1 = pos1
    lat2, lon2 = pos2
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6371.0 # Radius of Earth in kilometers
    return R * c # Distance in kilometers


def estimate_flight_time(pos1, pos2, speed_kmh=900):
    distance = haversine(pos1, pos2)
    time_hours = distance / speed_kmh
    return time_hours

# SCHEDULING

In [86]:
import random
from pytz import timezone
from datetime import datetime

def calculate_jet_lag(coord1, coord2):
    # Approximate time zone difference using pytz
    tz_dict = {
        'South Korea': 'Asia/Seoul',
        'Japan': 'Asia/Tokyo',
        'New Zealand': 'Pacific/Auckland',
        'Australia': 'Australia/Sydney'
    }
    
    # Find the countries corresponding to the coordinates
    country1 = next(country for country, loc in locationdict.items() if loc == coord1)
    country2 = next(country for country, loc in locationdict.items() if loc == coord2)
    
    tz1 = timezone(tz_dict[country1])
    tz2 = timezone(tz_dict[country2])
    
    now = datetime.now()
    offset1 = tz1.utcoffset(now).total_seconds() / 3600
    offset2 = tz2.utcoffset(now).total_seconds() / 3600
    
    return abs(offset1 - offset2)

def create_round_robin_schedule(countries, countriesratings, locationdict):
    def is_valid_schedule(schedule):
        for country, matches in schedule.items():
            home_count = 0
            away_count = 0
            consecutive_rating_diff = 0
            last_rating_diff = 0

            for match in matches:
                opponent, is_home = match
                rating_diff = abs(countriesratings[country] - countriesratings[opponent])

                # Check home/away constraints
                if is_home:
                    home_count += 1
                    away_count = 0
                else:
                    away_count += 1
                    home_count = 0

                if home_count > 2 or away_count > 2:
                    return False

                # Check rating difference constraints
                if rating_diff > 200:
                    if last_rating_diff > 200:
                        consecutive_rating_diff += 1
                    else:
                        consecutive_rating_diff = 1
                else:
                    consecutive_rating_diff = 0

                if consecutive_rating_diff > 2:
                    return False

                last_rating_diff = rating_diff

        return True

    def calculate_travel_and_jet_lag(schedule):
        total_distance = 0
        total_jet_lag = 0
        for country, matches in schedule.items():
            current_location = locationdict[country]
            for match in matches:
                opponent, is_home = match
                if not is_home:
                    opponent_location = locationdict[opponent]
                    total_distance += haversine(current_location, opponent_location)
                    total_jet_lag += calculate_jet_lag(current_location, opponent_location)
                    current_location = opponent_location
        return total_distance, total_jet_lag

    # Generate initial schedule
    schedule = {country: [] for country in countries}
    all_matches = [(c1, c2) for i, c1 in enumerate(countries) for c2 in countries[i + 1:]]
    random.shuffle(all_matches)

    for c1, c2 in all_matches:
        if len(schedule[c1]) <= len(schedule[c2]):
            schedule[c1].append((c2, True))  # c1 is home
            schedule[c2].append((c1, False))  # c2 is away
        else:
            schedule[c1].append((c2, False))  # c1 is away
            schedule[c2].append((c1, True))  # c2 is home

    # Optimize schedule
    while not is_valid_schedule(schedule):
        random.shuffle(all_matches)
        schedule = {country: [] for country in countries}
        for c1, c2 in all_matches:
            if len(schedule[c1]) <= len(schedule[c2]):
                schedule[c1].append((c2, True))
                schedule[c2].append((c1, False))
            else:
                schedule[c1].append((c2, False))
                schedule[c2].append((c1, True))

    # Minimize travel distance and jet lag
    best_schedule = schedule
    best_distance, best_jet_lag = calculate_travel_and_jet_lag(schedule)

    for _ in range(1000):  # Try 1000 random shuffles to minimize travel distance and jet lag
        random.shuffle(all_matches)
        schedule = {country: [] for country in countries}
        for c1, c2 in all_matches:
            if len(schedule[c1]) <= len(schedule[c2]):
                schedule[c1].append((c2, True))
                schedule[c2].append((c1, False))
            else:
                schedule[c1].append((c2, False))
                schedule[c2].append((c1, True))

        if is_valid_schedule(schedule):
            distance, jet_lag = calculate_travel_and_jet_lag(schedule)
            if distance + jet_lag < best_distance + best_jet_lag:
                best_schedule = schedule
                best_distance = distance
                best_jet_lag = jet_lag

    return best_schedule

# Generate the schedule
schedule = create_round_robin_schedule(countries, countriesratings, locationdict)

# Print the schedule and travel routes
for country, matches in schedule.items():
    print(f"{country}:")
    current_location = locationdict[country]
    for opponent, is_home in matches:
        location = "Home" if is_home else "Away"
        if not is_home:
            elo_diff = abs(countriesratings[country] - countriesratings[opponent])
            opponent_location = locationdict[opponent]
            distance = haversine(current_location, opponent_location)
            jet_lag = calculate_jet_lag(current_location, opponent_location)
            print(f"  vs {opponent} ({location}) - Distance: {distance:.2f} km, Jet Lag: {jet_lag:.2f} hours, Elo Diff: {elo_diff}")
            current_location = opponent_location
        else:
            elo_diff = abs(countriesratings[country] - countriesratings[opponent])
            print(f"  vs {opponent} ({location}), Elo Diff: {elo_diff}")


            # TLDR: This model generates a round-robin schedule for countries, ensuring constraints like home/away balance, rating differences, and minimizing travel distance and jet lag. It uses optimization and validation to create the best possible schedule.
            # Optimization and Validation Explanation:

            # 1. Validation:
            # The `is_valid_schedule` function ensures that the generated schedule adheres to the constraints:
            # - No team plays more than 2 consecutive home or away matches.
            # - No team faces opponents with an Elo rating difference greater than 200 for more than 2 consecutive matches.
            # This function iterates through each team's matches and checks these constraints. If any constraint is violated, the schedule is deemed invalid.

            # 2. Optimization:
            # The optimization process aims to minimize the total travel distance and jet lag for all teams.
            # - The `calculate_travel_and_jet_lag` function computes the total travel distance and jet lag for a given schedule.
            # - The initial schedule is generated randomly, and then the code attempts to improve it by shuffling the matches and regenerating the schedule.
            # - For each new schedule, the code checks its validity using `is_valid_schedule`. If valid, it calculates the travel distance and jet lag.
            # - The schedule with the lowest combined travel distance and jet lag is retained as the best schedule.

            # The process involves 1000 random shuffles to explore different possible schedules and find the optimal one. This approach balances computational efficiency with the quality of the solution.
            # Example usage:
            print("Optimized schedule created successfully.")

KeyboardInterrupt: 

In [ ]:
import folium
from folium import PolyLine

# Change the country to Australia
selected_country = "New Zealand"

# Create a folium map centered around Australia
map_center = locationdict[selected_country]
game_map = folium.Map(location=map_center, zoom_start=4)

# Add markers and lines for each game in the schedule for "Australia"
current_location = locationdict[selected_country]
for team, is_home in schedule[selected_country]:
    if not is_home:  # Only plot away games
        opponent_location = locationdict[team]
        # Add a marker for the opponent
        folium.Marker(
            location=opponent_location,
            popup=f"Opponent: {team}",
            icon=folium.Icon(color="blue", icon="info-sign"),
        ).add_to(game_map)
        # Add a line with an arrow to indicate travel
        PolyLine(
            locations=[current_location, opponent_location],
            color="blue",
            weight=2,
            opacity=0.8,
            tooltip=f"Travel to {team}",
        ).add_to(game_map)
        current_location = opponent_location

# Add a marker for the starting location (Australia)
folium.Marker(
    location=locationdict[selected_country],
    popup=f"{selected_country} (Home)",
    icon=folium.Icon(color="green", icon="home"),
).add_to(game_map)

# Display the map
game_map